Code to put together a crosswalk between NC Species of Greatest Conservation Need and GAP species.  The end goal is to link SGCN names to GAP species codes, as those codes are in the GAP habitat map names.

A first step is to go to https://www1.usgs.gov/csas/swap/state_list.html#state=North%20Carolina and download a list (csv file) of NC SGCN species.

Import packages:

In [93]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jul 13 11:21:43 2018 by nmtarr
"""
import sys
sys.path.append('C:/Program Files (x86)/ArcGIS/Desktop10.4/bin64')
sys.path.append('C:/Program Files (x86)/ArcGIS/Desktop10.4/ArcPy')
sys.path.append('C:/Program Files (x86)/ArcGIS/Desktop10.4/ArcToolBox/Scripts')
sys.path.append('P:/Proj3/USGap/Scripts/GAPProduction')
sys.path.append('P:/Proj3/USGap/Scripts/')
import gapproduction as gp
import gapconfig as config
import pandas as pd
pd.set_option('display.width', 2000)
import pickle

Read the csv as a pandas data frame and filter out everything but birds, mammals, amphibians, and reptiles.  Also remove species not on the 2015 SGCN list.

In [94]:
# Location of USGS NCSGCN list
sgcnCSV = "T:/SALCC/ncsgcn.csv"
BCBlist = pd.read_csv(sgcnCSV, sep="|")

# Filter out non-tetrapods
BCBlist2 = BCBlist[BCBlist['Taxonomic Group'].isin(['Birds', 'Reptiles', 
                   'Amphibians', 'Mammals'])]
print(BCBlist2.head(n=20))

# Filter out non-2015 species
BCBlist3 = BCBlist2[BCBlist2['Occurs in 2015 Data'] == 'yes']

# Save copy of data frame as csv
BCBlist3.to_csv('T:/SALCC/ncsgcn2015TetrapodsBCB.csv')

   Scientific Name Reported in State SWAP               Common Name Occurs in 2005 Data Occurs in 2015 Data Taxonomic Group Taxonomic Match Method
0                      Accipiter cooperii             Cooper's Hawk                 yes                  no           Birds            Exact Match
1                      Accipiter striatus        Sharp-shinned Hawk                 yes                  no           Birds            Exact Match
5                       Aegolius acadicus     Northern Saw-whet Owl                 yes                 yes           Birds            Exact Match
14             Alligator mississippiensis        American Alligator                 yes                 yes        Reptiles            Exact Match
17                       Ambystoma mabeei        Mabee's salamander                 yes                 yes      Amphibians            Exact Match
18                    Ambystoma maculatum        Spotted Salamander                 yes                  no      Amphi

Find records that match (sgcn to gap) on scientific name and record the gap code in the data frame when there's a match

In [95]:
# Get a species code dictionary
gapcodeDict = gp.gapdb.Dict_SciNameToCode()

# Use scientific names to look up GAPcodes, note if there isn't a match
combinedList = BCBlist3.copy()
combinedList.set_index('Scientific Name Reported in State SWAP', inplace=True)
for x in combinedList.index:
    try:
        code = gapcodeDict[x]
    except:
        code = 'no_GAP_match'
    combinedList.loc[x, 'GAPCode'] = code
combinedList.reset_index(inplace=True)
print(combinedList.head())

!!!completedModels Option doesn't work because the SppDB is not up to date!!!
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
Pseudacris regilla
Rana aurora
Taricha granulosa
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
Chasiempis sandwichensis
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
Non

See if records that didn't match based on scientific name will match on common name, and record the gap code when they do.

In [96]:
# Make a common name species code dictionary
d = {}
l = gp.gapdb.ListAllSpecies()
for i in l:
    try:
        com = gp.gapdb.NameCommon(i)
        if com in d.iterkeys():
            if i.endswith('x'):
                d[com] = i
            else:
                pass
        else:
            d[com] = i
    except:
        pass

combinedList.set_index('Common Name', inplace=True)
for x in combinedList.index:
    if combinedList.loc[x, 'GAPCode'] == 'no_GAP_match':
        try:
            code = d[x]
        except:
            code = 'no_GAP_match'
        combinedList.loc[x, 'GAPCode'] = code
        
combinedList.reset_index(inplace=True)

!!!completedModels Option doesn't work because the SppDB is not up to date!!!


For species that had a match (finding a gap code was possible), fill out columns for comon name, scientific name, and the ITIS TSN that is associated with the gap species code in the GAP databases.
www.itis.gov

In [97]:
combinedList.set_index(['GAPCode'], inplace=True)
for x in combinedList.index:
    try:
        combinedList.loc[x, 'GAP_common_name'] = gp.gapdb.NameCommon(x)
        combinedList.loc[x, 'GAP_sci_name'] = gp.gapdb.NameSci(x)
        combinedList.loc[x, 'ITIS_TSN'] = gp.gapdb.Crosswalk(x)[2]
    except:
        pass
combinedList.reset_index(inplace=True)
combinedList.to_csv("T:/SALCC/ncsgcn-gap_crosswalk.csv")
print(combinedList.head())

  GAPCode               Common Name Scientific Name Reported in State SWAP Occurs in 2005 Data Occurs in 2015 Data Taxonomic Group Taxonomic Match Method           GAP_common_name                GAP_sci_name ITIS_TSN
0  bNSWOx     Northern Saw-whet Owl                      Aegolius acadicus                 yes                 yes           Birds            Exact Match     Northern Saw-whet Owl           Aegolius acadicus   177942
1  rAMALx        American Alligator             Alligator mississippiensis                 yes                 yes        Reptiles            Exact Match        American Alligator  Alligator mississippiensis   551771
2  aMBSAx        Mabee's salamander                       Ambystoma mabeei                 yes                 yes      Amphibians            Exact Match        Mabee's Salamander            Ambystoma mabeei   173600
3  aMOSAx           Mole Salamander                   Ambystoma talpoideum                 yes                 yes      Amphibians  

Show the species without an obvious match to GAP species list.

In [103]:
nonmatches = combinedList[combinedList['GAPCode'] == 'no_GAP_match'].set_index(['Scientific Name Reported in State SWAP'])
print(nonmatches.drop(columns=['Occurs in 2015 Data', 'Occurs in 2005 Data', 'Taxonomic Group', 'GAPCode', 
                               'Taxonomic Match Method'], axis=1))

                                                                             Common Name GAP_common_name GAP_sci_name ITIS_TSN
Scientific Name Reported in State SWAP                                                                                        
Apalone spinifera aspera                                      Gulf Coast Spiny Softshell            None         None         
Apalone spinifera spinifera                                      Eastern Spiny Softshell            None         None         
Balaenoptera physalus                                                          Fin Whale            None         None         
Calidris pusilla                                                  Semipalmated Sandpiper            None         None         
Cemophora coccinea copei                                          Northern Scarlet Snake            None         None         
Corynorhinus rafinesquii macrotis        Rafinesque's Big-eared Bat - Coastal Plain Pop.            None       

    I thought it might be helpful to have a table of all the GAP species and some of their taxonomic information.

In [99]:
UCs = pickle.load(open('T:/GAP/Data/AllUCs.pkl', 'r'))
gapList = pd.DataFrame(index=UCs)
for x in gapList.index:
    if gp.gapdb.NameSci(x) != None:
        gapList.loc[x, 'scientific_name'] = gp.gapdb.NameSci(x)
        gapList.loc[x, 'common_name'] = gp.gapdb.NameCommon(x)
        gapList.loc[x, 'ITIS_TSN'] = gp.gapdb.Crosswalk(x)[2]
gapList.to_csv("T:/SALCC/GAPspecies.csv")
gapList

,scientific_name,common_name,ITIS_TSN
aABSAx,Eurycea waterlooensis,Austin Blind Salamander,668294
aACSSx,Plethodon chlorobryonis,Atlantic Coast Slimy Salamander,208282
aADSAx,Desmognathus apalachicolae,Apalachicola Dusky Salamander,208266
aAGTOx,Anaxyrus nelsoni,Amargosa Toad,773526
aAMBUx,Lithobates catesbeianus,American Bullfrog,775084
aAMDSx,Desmognathus ochrophaeus,Allegheny Mountain Dusky Salamander,173641
aAMTOx,Anaxyrus americanus,American Toad,773511
aARSAx,Aneides lugubris,Arboreal Salamander,173703
aARTOx,Anaxyrus californicus,Arroyo Toad,773514
aAZTOx,Anaxyrus microscaphus,Arizona Toad,773525
